In [1]:
import numpy as np 
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from nltk import word_tokenize
from sklearn.metrics import f1_score
import tensorflow as tf

In [2]:
# SUMMARY
# Epochs - 7
# Total training run time - 981s
# Training final epoch loss - 0.4467
# Training final epoch accuracy - 0.8261
# Training final epoch val_loss - 0.7810
# Training final epoch val_accuracy - 0.7162
# Test loss - 0.7242
# Test accuracy - 0.7222
# Macro F1 Score - 0.6152
# Markov generated accuracy - 0.8628
# RNN generated accuracy - 0.9651
# Markov syllable generated accuracy - 0.9173

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# All Rock
rock1 = open('/content/drive/My Drive/Colab Notebooks/NewAllRock.txt', 'r').read()
rock = ''.join([i for i in rock1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Pop
pop1 = open('/content/drive/My Drive/Colab Notebooks/NewAllPop.txt', 'r').read()
pop = ''.join([i for i in pop1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Country
country1 = open('/content/drive/My Drive/Colab Notebooks/NewAllCountry.txt', 'r').read()
country = ''.join([i for i in country1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Rap
rap1 = open('/content/drive/My Drive/Colab Notebooks/AllLyrics.txt', 'r').read()
rap = ''.join([i for i in rap1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')

In [5]:
# create samples of 10 words each for each genre - this is our estimate length of line
SONG_LENGTH = 10
# Rock
Rock = [rock[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(rock)/int(SONG_LENGTH)))]
# Country
Country = [country[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(country)/int(SONG_LENGTH)))]
# Pop
Pop = [pop[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(pop)/int(SONG_LENGTH)))]
# Rap
Rap = [rap[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(rap)/int(SONG_LENGTH)))]

In [6]:
ds_rock = [' '.join(Rock[i]) for i in range(len(Rock))]
ds_country = [' '.join(Country[i]) for i in range(len(Country))]
ds_pop = [' '.join(Pop[i]) for i in range(len(Pop))]
ds_rap = [' '.join(Rap[i]) for i in range(len(Rap))]

In [7]:
# rock 0, country 1, pop 2, rap 3
ds_ro = []
genre = 0
for sample in ds_rock:
  ds_ro.append([genre, sample])

ds_co = []
genre = 1
for sample in ds_country:
  ds_co.append([genre, sample])

ds_po = []
genre = 2
for sample in ds_pop:
  ds_po.append([genre, sample])

ds_ra = []
genre = 3
for sample in ds_rap:
  ds_ra.append([genre, sample])

ds = ds_ro+ds_co+ds_po+ds_ra

ds = np.array(ds)
print('Genres: ', ds[:, 0])
print('Lyrics: ', ds[:, 1])

Genres:  ['0' '0' '0' ... '3' '3' '3']
Lyrics:  ['theres a lady whos sure all that glitters is gold'
 'and shes buying a stairway to heaven when she gets'
 'there she knows if the stores are all closed with' ...
 'kick me when im down but im up again scorchin'
 'hot forcin my way up in the door to kill'
 'the bullpoop like a matador keep your hands high what']


In [8]:
# max words we use
maxwords = 50000
# max words per lyric (should be 200 anyway as set up)
maxlyricwords = 200
# tokenize here
embedding_dimension = 100
tokenizer = Tokenizer(num_words=maxwords, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(ds[:, 1])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 17790 unique tokens.


In [9]:
X = tokenizer.texts_to_sequences(ds[:, 1])
X = pad_sequences(X, maxlen=maxlyricwords)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (58448, 200)


In [10]:
Y = pd.get_dummies(ds[:, 0])
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (58448, 4)


In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(52603, 200) (52603, 4)
(5845, 200) (5845, 4)


In [12]:
model = Sequential()
model.add(Embedding(maxwords, embedding_dimension, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 26
batch_size = 256
load_model = False
save_path = 'model_save_RNN/cp.ckpt'

if not load_model:
    model_callback_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_callback_saving = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, save_weights_only=True, verbose=1)

    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1, callbacks=[model_callback_stopping, model_callback_saving])
else:
    model.load_weights(save_path)
    print('Restored Model')

# On previous run Epoch 1 was 205s, 1.5hrs has 5400s so will run for [5400/205]=26 epochs on GPU

Epoch 1/26
185/185 [==============================] - 147s 768ms/step - loss: 1.1425 - accuracy: 0.5444 - val_loss: 0.8696 - val_accuracy: 0.6497

Epoch 00001: saving model to model_save_RNN/cp.ckpt
Epoch 2/26
185/185 [==============================] - 139s 750ms/step - loss: 0.7756 - accuracy: 0.6878 - val_loss: 0.7698 - val_accuracy: 0.6907

Epoch 00002: saving model to model_save_RNN/cp.ckpt
Epoch 3/26
185/185 [==============================] - 138s 746ms/step - loss: 0.6353 - accuracy: 0.7513 - val_loss: 0.7472 - val_accuracy: 0.7073

Epoch 00003: saving model to model_save_RNN/cp.ckpt
Epoch 4/26
185/185 [==============================] - 139s 750ms/step - loss: 0.5476 - accuracy: 0.7875 - val_loss: 0.7378 - val_accuracy: 0.7149

Epoch 00004: saving model to model_save_RNN/cp.ckpt
Epoch 5/26
185/185 [==============================] - 139s 753ms/step - loss: 0.5079 - accuracy: 0.8037 - val_loss: 0.7517 - val_accuracy: 0.7122

Epoch 00005: saving model to model_save_RNN/cp.ckpt
Epoch

In [13]:
# Test set
accr = model.evaluate(X_test,Y_test)

183/183 [==============================] - 8s 43ms/step - loss: 0.7242 - accuracy: 0.7222


In [14]:
# Macro F1 score
y_pred = np.around(model.predict(X_test))
f1_score(Y_test, y_pred, average='macro')

0.6152372064022109

In [15]:
def classify_string(input):
  lyric = [str(input)]
  seq = tokenizer.texts_to_sequences(lyric)
  padded = pad_sequences(seq, maxlen=maxlyricwords)
  pred = model.predict(padded)
  labels = ['rock', 'country', 'pop', 'rap']
  print(labels[np.argmax(pred)])

In [16]:
def classify_list(input):
  for lyric in input:
    classify_string(lyric)

In [17]:
def classify_perc(input):
  def cla(input):
    lyric = [str(input)]
    seq = tokenizer.texts_to_sequences(lyric)
    padded = pad_sequences(seq, maxlen=maxlyricwords)
    pred = model.predict(padded)
    labels = ['rock', 'country', 'pop', 'rap']
    return str(labels[np.argmax(pred)])
  results = list(map(cla, input))
  return (results.count('rap'))/400

In [25]:
def classify_str(input):
  lyric = [str(input)]
  seq = tokenizer.texts_to_sequences(lyric)
  padded = pad_sequences(seq, maxlen=maxlyricwords)
  pred = model.predict(padded)
  labels = ['rock', 'country', 'pop', 'rap']
  return labels[np.argmax(pred)]

In [18]:
# markov generated lyrics sample
lyrics = ['Bumpin i meant for you call my ninja like',
 'Biz dont take their baby mommas ninja frick you nasty boy you',
 'Shifty sticks and pray and flee the frick all of you',
 'Glocks but all ill die slow',
 'Wondering if im askin blunt sip champagne range rover been outside for',
 'And youre so take that crown two pounds you know',
 'Publishing i thought i get witcha can i could cop',
 'Miss the more cause you in the right one',
 'Onyx and them hoes i love',
 'Gat call me puff daddy biggie gots ta like',
 'Everything around me shit b***** in ya imma stay yappin when',
 'Hum all about fingers in the loot im',
 'Rollem up heard whos this yeah keep on top sky is',
 'Drunk of ninjaz from now drop to',
 'Declinin windin like flypaper neighbor slow down',
 'Expensive cars i tote my crew i only got enough heart',
 'Lame dudes whos next move but the drugs to spit phrases thatll',
 'Guy well its cool and your poop so hard to',
 'Clap wit my life in ma little nasty boy',
 'Dial you should too much better man played',
 'Lali like that you frick doin all mcs have']

classify_list(lyrics)

rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap


In [19]:
# lstm generated lyrics sample
lyrics2 = ['in the veins hard to explain how i maintain', 
  'to put my back in the house so i can i wanna flaunt you thats right', 
  'with the grime of my ninja frick',
  'with the ds crept in blastin him you dont want to slit the clits alot',
  'used to lick the clits a lot of problems never be the beamer with the goldie sound',
  'like a steelo not my steelo oh no thats not my my steelo oh i steelo not my steelo oh no',
  'thats not my no steelo bust my no dough day but this sittin bodies not my']

classify_list(lyrics2)

pop
pop
rap
rap
rap
rap
rap


In [20]:
# Our two tests for Markov generated and RNN generated lyrics are below - each 400 lines worth
# Lyrics included so that they can be seen

In [26]:
# Markov
markov_lyrics = ['Come here sit on a spot up out with em',
 'Put the game ninja cause please believe it was way',
 'Brought the golf hat the one eight seven kidnaps and',
 'Guns  yall a read yall a gangsta bedtime story',
 'You better have you need a gangsta bedtime story by',
 'Ever slapped a gangsta bedtime story  great scotts its',
 'Here sit on a kit kat  great scotts its',
 'Bedtime story by saying goodnight  yall get em frick',
 'Upon a trip up in town my ninja even if',
 'Do my ninja caught up on a trip up to',
 'All the downlow oh once upon a trip up on',
 'Do nothing to get a gangsta bedtime story  yall',
 'Grease strikes you need a gangsta bedtime story  and',
 'To stay on some traffic behind some traffic behind some',
 'Can ride on some traffic behind some traffic behind some',
 'What time it was way to him cause please believe',
 'Couldnt do my lap  to him cause please believe',
 'S-n double o-p fa sho i had to real i',
 'Gotta end this muthafricka cuz i had to real i',
 'In the women stayed true to get your b***** aint',
 'Chest he was we got thugs cons drugs and i',
 'Your sack and jacks i gotta end this story ',
 'Stick ninja and then i gotta end this story ',
 'Ride on deck so i ran up in his hands',
 'Eatin on deck so i ran up in his hands',
 'Spot up in his hands in this ninja and this',
 'End this story by saying goodnight  and jacks i',
 'Thugs cons drugs and your grip or better yet strapped',
 'Brought snaps to mack your grip or better yet strapped',
 'Today he ran up in some traffic behind some hood',
 'Muthafrickin vietnam vet riding on some traffic behind some hood',
 'Kids looked up in some ninjas for heater  could',
 'From the one eight seven kidnaps and doves have you',
 'Vet riding on this ninja im that ninja raps to',
 'Way to the little homie hit that ninja raps to',
 'Drop you read us a bedtime story by saying goodnight',
 'Strikes you need a bedtime story by saying goodnight ',
 'Sack before i shot him because he tried to do',
 'Sack before i do my ninja even if we cool',
 'Its hot anybody can go down my ash tray get',
 'Sho i brought snaps to do my ash tray get',
 'Slapped a steak eatin on this ninja even if we',
 'Women stayed true to stay on this ninja even dimes',
 'Homie hit that big rap name ninja caught up on',
 'Floor with me we claiming everything ninja caught up to',
 'One eight seven kidnaps and go get back and your',
 'Skandelous raps to the black poker sack and your b*****',
 'Brought the dpg and jacks i came out  yall',
 'Stranger to him but i gotta end this skandelous raps',
 'Danger aint no stranger to him cause hes like me',
 'Vietnam vet riding on this ninja cause hes like me',
 'Stainless steel and guns  okay check it was wearing',
 'Lighter  and doves have a time it was wearing',
 'Show these ninjas for a clip  okay check it',
 'Whassup  and guns  alright  okay check it',
 'Have you need a kit kat  come here sit',
 'Frick em lets go get back  come here sit',
 'Cut throat on the curl back and jacks i brought',
 'Thugs cons drugs and this ninja and jacks i brought',
 'Who brought the black poker sack before i gotta end',
 'True to the black poker sack before i gotta end',
 'Didnt care peck he ran up on deck so fly',
 'Dont know why but i ran up on deck so',
 'Cool with his vision gettin blurry but hes like a',
 'Rat grease strikes you need a ninja even dimes and',
 'Cuz i came out with em frick em lets go',
 'Tray get back and all the game like me to',
 'Cuz i shot him because he ran up on the',
 'Vision gettin hot today he ran up on the lbc',
 'Could get a b***** aint no get my ninja s-n',
 'Stayed true to him in his hands in some ninjas',
 'Or better yet strapped a stick ninja caught up to',
 'Need a read yall a stick ninja caught up to',
 'Me lets go get my ninja like a bedtime story',
 'Og from the lbc there lived a bedtime story by',
 'Town my ash tray get my ninja and all the',
 'Here sit back wishing for that ninja and all they',
 'Eatin on him in his hands in his hands in',
 'Skandelous raps the golf hat the air his hands in',
 'Pat your grip or better have a muthafrickin vietnam vet',
 'Your grip or better yet strapped a muthafrickin vietnam vet',
 'Before i shot him all even dimes and this ninja',
 'Lap  to him cause hes like a stick ninja',
 'Because he fell to do my set no stranger to',
 'Air his hands in town my set no stranger to',
 'Brought the air his chest he was way to do',
 'Shot him in his chest he was way to do',
 'Smack for that im that sit on this story by',
 'Ash tray get a steak eatin on this story ',
 'That sit back  okay check it it it was',
 'Riding on some ninjas for a time in this ninja',
 'Em lets go get a gangsta bedtime story  okay',
 'Ran up to him police tried to do my lap',
 'Story  and i turn around and the downlow oh',
 'Stainless steel and your b***** aint poop the downlow oh',
 'Stay on the little kids looked up on the hood',
 'This ninja who brought snaps to him in some hood',
 'Saying goodnight  could you read us a trip up',
 'Wearing slack i do my ninja like a trip up',
 'Poop the game ninja who brought the dpg and guns',
 'Dont know why but you better have you read us',
 'Cut throat on the ride down my lap  okay',
 'No warning shots on some hood gettin hot anybody can',
 'Better yet strapped a time in his hands in the',
 'Tried to the floor with his hands in the dpg',
 'Great scotts its hot today he ran up with me',
 'Great scotts its hot today he ran up in the',
 'Warning shots on the afro back and snatch your back',
 'Og from the afro back  and pat your sack',
 'Wearing slack i shot him because he was wearing slack',
 'Warning shots on this ninja you ever slapped a ninja',
 'Hated on some hood gettin blurry but i brought the',
 'Wearing slack i dont know why but i brought the',
 'Who brought snaps to show these ninjas what time in',
 'So i had to show these ninjas what time it',
 'His hands in the little kids looked up out of',
 'Dpg and the little kids looked up out  to',
 'For heater  great scotts its hot today he fell',
 'Kat  and the hood gettin hot today he fell',
 'Like stainless steel and i didnt care peck he tried',
 'On this muthafricka cuz i didnt care peck he tried',
 'Whassup whassup  yall a clip  we cool with',
 'At all the air his hands in the floor with',
 'Claiming everything ninja disrepectin my ninja disrepectin my lap ',
 'Riding on this ninja disrepectin my ninja disrepectin my lap',
 'Of g two seater you out  we can drop',
 'Heater  we can drop you could get a spot',
 'Shot him in his neck shooting like that im a',
 'Trip up in his neck shooting like that im a',
 'Tried to mack your game ninja cause hes like stainless',
 'Millimeter for that young ninja even if we can ride',
 'Raps the little kids looked up on deck so fly',
 'Could you better have a trip up on deck so',
 'Just so i do him in town my thang way',
 'Have a muthafrickin vietnam vet riding on my ash tray',
 'Him all the floor with em lbc there lived a',
 'Before i ran up with em lbc there lived a',
 'Goodnight  and jacks i ran up in this ninja',
 'Me lets go get a spot up in this ninja',
 'Danger aint poop the afro back wishing for that young',
 'Neck shooting like that sit back wishing for that young',
 'At all the afro back wishing for that young ninja',
 'Shots on a muthafrickin nine millimeter for that young ninja',
 'Wishing for nuthin at all they hated on deck so',
 'Young ninja and all they hated on deck so fly',
 'Vet riding on a time in the game ninja you',
 'Got thugs cons drugs and put the game ninja you',
 'Riding on this story by saying goodnight  great scotts',
 'Eatin on this story by saying goodnight  great scotts',
 'Even dimes and then i came out of g two',
 'Aint no get a trip up out of g two',
 'It can go get a steak eatin on this ninja',
 'Check it out  come here sit on this ninja',
 'Why but couldnt do him but hes like me we',
 'Here sit on this ninja cause hes like me lets',
 'Bedtime story  to the golf hat the women stayed',
 'The afro back  to him all the women stayed',
 'Know why but i had to the ride down out',
 'There lived a b***** aint poop the ride down out',
 'Stranger to a clip  and then i ran up',
 'Shot him all even dimes and then i ran up',
 'Who brought the little kids looked up out ninjas what',
 'Back wishing for heater  could you out ninjas was',
 'Turn around and guns  okay check it out of',
 'Wishing for a clip  okay check it out of',
 'Fa sho i ran up on this muthafricka cuz i',
 'Floor with no warning shots on this muthafricka cuz i',
 'Eight seven kidnaps and snatch your sack and all they',
 'Turn around and snatch your back  and all the',
 'True to the curl back  im a ninja s-n',
 'Vision gettin hot today he was wearing slack i had',
 'Today he fell to him cause please believe it was',
 'Vietnam vet riding on him cause please believe it can',
 'B***** aint no warning shots on the little kids looked',
 'Police tried to a og from the little kids looked',
 'Brought the little homie hit that ninja raps the lbc',
 'Stay on the little homie hit that big rap name',
 'On a og from the game ninja and snatch your',
 'Golf hat the one eight seven kidnaps and pat your',
 'Could you need a time in his chest he fell',
 'Sit on the air his neck shooting like me lets',
 'End this ninja and your grip or better yet strapped',
 'Afro back and snatch your grip or better yet strapped',
 'Know why but you need a smack for that im',
 'We cool with no get a smack for that big',
 'Fell to the one eight seven kidnaps and then i',
 'Cons drugs and all the curl back and then i',
 'Can drop you out of g two seater you could',
 'Time it out of g two seater you out ',
 'Turn around and jacks i came out  okay check',
 'Down out  and go down out  okay check',
 'These ninjas what time in town my set no get',
 'Cons drugs and snatch your b***** aint poop the one',
 'Riding on my ninja and put the women stayed true',
 'Got thugs cons drugs and put the women stayed true',
 'Ever slapped a muthafrickin vietnam vet riding on a kit',
 'Ever slapped a muthafrickin vietnam vet riding on a clip',
 'Before i turn around and your grip or better yet',
 'Because he fell to mack your grip or better yet',
 'Your game like a read yall a kit kat ',
 'Oh once upon a read yall a kit kat ',
 'He ran up out with me lets go down my',
 'Steel and doves have you out with me we cool',
 'Real i shot him in his neck shooting like stainless',
 'Thang way to the air his neck shooting like stainless',
 'Lbc there lived a gangsta bedtime story  okay check',
 'Curl back  im a bedtime story  okay check',
 'Show these ninjas what time it can ride down my',
 'Clip  im a time it can ride down my',
 'Cons drugs and then i gotta end this muthafricka cuz',
 'Even dimes and jacks i gotta end this muthafricka cuz',
 'Like stainless steel and all the black poker sack and',
 'Homie hit that ninja raps the black poker sack and',
 'Doves have a og from the black poker sack and',
 'Wearing slack i had to the black poker sack and',
 'Turn around and go see take a b***** aint no',
 'Muthafricka cuz i brought the dpg and jacks i do',
 'Og from the game ninja you need a og from',
 'Snaps to the game ninja you need a og from',
 'Go down out ninjas was wearing slack i didnt care',
 'In the women stayed true to real i didnt care',
 'Afro back  we cool with his chest he tried',
 'Sack before i do him in his chest he tried',
 'Blurry but you read us a read yall get back',
 'See take a smack for a read yall get a',
 'Snatch your game ninja im that young ninja who brought',
 'A muthafrickin nine millimeter for that young ninja who brought',
 'They hated on some traffic behind some ninjas was wearing',
 'Vet riding on some traffic behind some ninjas was wearing',
 'Floor with me we claiming everything ninja caught up in',
 'Young ninja im that im that ninja caught up with',
 'Behind some traffic behind some ninjas what time it it',
 'Stayed true to show these ninjas what time it it',
 'Disrepectin my ash tray get em lets go down out',
 'Steel and this story  and go down out ',
 ' to do him all even if we got thugs',
 'That big rap name ninja even if we got thugs',
 'Kidnaps and all the floor with em lets go get',
 'The floor with his neck shooting like me to the',
 'Cool with no get my ninja like me to him',
 'Dont know i didnt care peck he fell to him',
 'Kit kat  okay check it was we cool with',
 'Heater  okay check it it was we cool with',
 'Dimes and i shot him police tried to the little',
 'Know i shot him because he fell to the little',
 'On this story  and pat your grip or better',
 'Name ninja even dimes and snatch your grip or better',
 'Came out of bullets so fly but you could get',
 'Pat your grip or better have you out of bullets',
 'Me lets go get your back and put the ride',
 'Clip  yall get back  and put the little',
 'From the golf hat the curl back and your b*****',
 'One eight seven kidnaps and guns  and pat your',
 'To the dpg and doves have you could you could',
 'Disrepectin my lighter  could you know why but you',
 'By saying goodnight  and guns  we cool with',
 'Game ninja raps to real i brought the floor with',
 'Didnt care peck he ran up with me lets go',
 'We claiming everything ninja caught up with me we got',
 'Curl back and all they hated on deck so i',
 'Wearing slack i brought the ride on deck so i',
 'Fly but couldnt do him because he was we can',
 'Women stayed true to him but you need a steak',
 'Its hot anybody can go get my ninja like me',
 'Cuz i shot him in the game ninja like me',
 'Aint no get a steak eatin on deck so i',
 'Strikes you need a steak eatin on deck so i',
 'Can go see take a og from the air his',
 'Kat  could get a bedtime story  we claiming',
 'Fly but couldnt do my lighter  alright  okay',
 'Game ninja disrepectin my ninja like that sit on a',
 'Throat on a stick ninja like a time it was',
 'Ninja disrepectin my ash tray get a time it was',
 'Take a og from the little kids looked up out',
 'Afro back and put the little kids looked up out',
 'B***** to the curl back and jacks i gotta end',
 'Can drop you ever slapped a b***** aint no stranger',
 'Thang way to stay on my set no get a',
 'Why but couldnt do my set no get my lap',
 'Poop the golf hat the golf hat the floor with',
 'Around and i brought the golf hat the floor with',
 'Came out with me lets go get back wishing for',
 'Yet strapped a spot up to get back wishing for',
 'Gotta end this muthafricka cuz i brought the ride down',
 'Town my lap  and put the ride down my',
 'Great scotts its hot anybody can go get a clip',
 'Snatch your back  and go get a clip ',
 'From the downlow oh once upon a og from the',
 'Because he ran up to a og from the lbc',
 'Stay on deck so fly but i brought the game',
 'Real i dont know why but i brought the game',
 'Kidnaps and snatch your game like that young ninja like',
 'Mack your grip or better yet strapped a ninja like',
 'Shots on this skandelous raps the lbc there lived a',
 'Ride down my ninja raps the lbc there lived a',
 'Golf hat the afro back and all even if we',
 'Hands in town my lap  and all even dimes',
 'At all the game ninja dip with em frick em',
 'Didnt care peck he tried to him in the little',
 'Aint no warning shots on some ninjas what time in',
 'Set no warning shots on some ninjas what time it',
 'S-n double o-p fa sho i shot him but hes',
 'S-n double o-p fa sho i shot him but hes',
 'Have you better yet strapped a og from the afro',
 'Us a spot up to a og from the afro',
 'Claiming everything ninja disrepectin my thang way to the dpg',
 'Kit kat  come here sit back and go get',
 'Tray get my lighter  and all the afro back',
 'Snatch your grip or better have a b***** to a',
 'Here sit on my set no stranger to stay on',
 'Vet riding on my ninja s-n double o-p fa sho',
 'There lived a spot up on the curl back and',
 'My set no warning shots on the curl back and',
 'Hood rat grease strikes you read us a time in',
 'I shot him in some traffic behind some traffic behind',
 'Throat on the afro back and pat your sack before',
 'Who brought the afro back and pat your sack before',
 'And i gotta end this ninja s-n double o-p fa',
 'Sit on him in this ninja s-n double o-p fa',
 'Cut throat on this story  alright alright alright alright',
 'Steel and guns  alright alright alright alright alright ',
 'So i ran up in this ninja im a ninja',
 'Ninjas what time in this ninja im that im a',
 'Check it out of g two seater you ever slapped',
 'Looked up out of g two seater you ever slapped',
 'Ran up out of g two seater you know i',
 'Drop you out of g two seater you know i',
 'And snatch your b***** aint poop the little homie hit',
 'Kat  yall a og from the little homie hit',
 'Everything ninja dip with me we got thugs cons drugs',
 'Jacks i came out of bullets so fly but you',
 'Skandelous raps to him but couldnt do him because he',
 'Gotta end this story  to do him because he',
 'Tray get a clip  im that ninja raps the',
 'Yet strapped a clip  im that big rap name',
 'Show these ninjas for nuthin at all the one eight',
 'Hat the black poker sack and all the one eight',
 'Whassup  alright alright  and go get my ninja',
 'At all the lbc in some traffic behind some ninjas',
 'Had to stay on a bedtime story  we got',
 'Had to real i brought the ride on deck so',
 'Homie hit that sit on my ninja who brought the',
 'Just so i brought the game ninja who brought the',
 'Seater you read yall a trip up with his neck',
 'Aint poop the little kids looked up in his chest',
 'Spot up out of bullets so fly but couldnt do',
 'Traffic behind some hood rat grease strikes you out of',
 'But couldnt do nothing to him in some ninjas what',
 'Behind some traffic behind some traffic behind some ninjas what',
 'Game ninja im that big rap name ninja disrepectin my',
 'Young ninja im that big rap name ninja disrepectin my',
 'O whassup whassup  im that im that sit on',
 'From the little homie hit that im that sit on',
 'Millimeter for nuthin at all they hated on the dpg',
 'Young ninja even dimes and all they hated on the',
 'Oh once upon a b***** to him in town my',
 'Big rap name ninja raps to him in town my',
 'Us a ninja s-n double o-p fa sho i ran',
 'Take a ninja s-n double o-p fa sho i ran',
 'Tray get my thang way to stay on some hood',
 'Because he ran up to danger aint poop the downlow',
 'Town my thang way to the dpg and the air',
 'Stay on the curl back and doves have a trip',
 'Take a clip  yall get a ninja dip with',
 'Jacks i came out ninjas was wearing slack i didnt',
 'Lap  yall a time in the game like stainless',
 'Shots on my ninja who brought the game like stainless',
 'Rap name ninja disrepectin my set no get your back',
 'His neck shooting like me lets go get your sack',
 'Kids looked up in town my lighter  could you',
 'Looked up in some hood rat grease strikes you out',
 'Eatin on a muthafrickin vietnam vet riding on the downlow',
 'Once upon a muthafrickin vietnam vet riding on the downlow',
 'Jacks i brought the women stayed true to him because',
 'His hands in his chest he tried to him because',
 'Air his chest he ran up with his neck shooting',
 'Lighter  yall a trip up in his neck shooting',
 'Doves have you ever slapped a muthafrickin nine millimeter for',
 'Fly but you better have a muthafrickin nine millimeter for',
 'Stainless steel and jacks i had to him because he',
 'Warning shots on this skandelous raps to him because he',
 'Have a steak eatin on some traffic behind some ninjas',
 'Stayed true to real i ran up in some ninjas',
 'To mack your b***** aint poop the lbc in town',
 'Warning shots on some traffic behind some hood gettin hot',
 'Like stainless steel and snatch your back wishing for heater',
 'Thugs cons drugs and pat your back wishing for heater',
 'Slapped a og from the ride down out of g',
 'Thang way cut throat on the ride down out of',
 'Snatch your back  could you know i shot him',
 'Some hood gettin blurry but you know i shot him',
 'Hood gettin hot anybody can drop you know why but',
 'Around and all even if we can drop you could',
 'Skandelous raps the ride down my set no get a',
 'Everything ninja dip with me lets go get a b*****',
 'Story  im a muthafrickin nine millimeter for a kit',
 'To do my ash tray get my set no warning',
 'Real i shot him in the floor with em lets',
 'Claiming everything ninja even if we cool with me lets',
 'Brought snaps to him because he fell to him police',
 'Sack and i shot him police tried to him police']


In [27]:
qp = map(classify_str, markov_lyrics)
list(qp).count('rap')

346

In [28]:
len(markov_lyrics)

401

In [29]:
346/401

0.8628428927680798

In [30]:
# RNN
RNN_lyrics = ['And i got to split ya',
'Then i get witcha can i get witcha',
'To all my queensbridge nixgas nixgas',
'Im a bad bad boy',
'When i was young in the gray uhhuh',
'Also known as your cherry they been through the waist',
'Now i dont know what they want from me',
'You nasty boy you nasty',
'I got the funk flow to make your drawers drop slow',
'What you want nixga what you what you want nixga',
'Lyin to pray for my downfall',
'Yes i got a gun on you',
'Get high get high get high',
'Call me evil at my bridge',
'But i dont care what nobody say',
'I got the funk flow to make your drawers drop slow',
'Smith you know what they want from me',
'frickin sayin',
'When i was young in the gray uhhuh',
'Jazze you get to get the new d*ck,'
'And i got to split ya',
'It aint no one but i got a d*ck and puffed me',
'And i got to split ya',
'Thats the nixga size in these karl kani jeans',
'Smoke a stone eggs and wednesdays',
'Off the hands high like youse a true player',
'Youre dead wrong',
'A notorious big',
'Biggie smalls the rap phenomenon',
'Drunk by a daily crackin',
'See i dont know what they want from me',
'I got the funk flow to make your drawers drop slow',
'Cause i get witcha can i get witcha',
'But i dont care what nobody say',
'I got the funk flow to make your drawers drop slow',
'Keep on pressin on',
'If you got a gun in your waist please dont shoot up the place',
'Beef is when your moms is poppa',
'Can i get witcha can i get witcha',
'To all my queensbridge nixgas nixgas',
'As i kiss your bum goodnight',
'As i kiss your bum goodnight',
'You nasty boy you nasty',
'Squeeze your clip hit the right one pass that sandwhiches i got to light one',
'Is the bad bad boy',
'Stepped to police when i proceed',
'Look at the streets to the phone to the door',
'Televisions believable believable believable',
'Verse the more money we come across',
'And i got to split ya',
'If you got a gun in your waist please dont shoot up the place',
'All the ladies if you hear me',
'What you want nixga what you what you want nixga',
'All the ladies if you hear me',
'Meshed on the bronx back to cali cali going back to cali',
'When i was young in the gray uhhuh',
'You nasty boy you nasty',
'Sex is all i get witcha can i get witcha',
'Matter of fact Im sick of blue',
'Sixtynine code',
'Ya aint no dough and squeeze and Im sniff me',
'Jm motherfrickers',
'Nobody gotta die',
'To all my queensbridge nixgas nixgas',
'Word to be over new york',
'Im a bad bad boy',
'My notorious big',
'Michelle i dont get witcha can i get witcha',
'You nasty boy you nasty',
'One in the chamber the streets that',
'And i got to split ya',
'Rubberband d havin nightmares of a melon of frickin',
'The weak or the strong who got it goin on',
'Make the cherokee to the frick to the d*ck',
'Time to be a worthless kid she damage and check it',
'I got the funk flow to make your drawers drop slow',
'All the ladies if you hear me',
'frickin sayin',
'Who rock around my d*ck than the waist please dont shoot to the thriller',
'What you want nixga what you what you want nixga',
'Its bone and biggie biggie',
'Just bone and biggie biggie',
'I got the funk flow to make your drawers drop slow',
'Nixga please be cool but they know we go',
'Sometimes your words just hypnotize me',
'Beef is when your moms is poppa',
'Im a bad bad boy',
'To all my queensbridge nixgas nixgas',
'I got the funk flow to make your drawers drop slow',
'You nasty boy you nasty',
'Smokin blunts in your passenger dream',
'Way it to the frick to the d*ck',
'Everything you be styling on you',
'Pull the gstring in your bed',
'Bought the cherokee to the crib unless they bonin',
'Still tote a purpose on the bladder and gentlemen with these elevated world',
'Only i get witcha can i get witcha',
'Why you wanna get with me',
'Gats and pray for my downfall',
'Dont stop Im not finished yet',
'Slugs missed ya',
'I got the funk flow to make your drawers drop slow',
'You nasty boy you nasty',
'Before i get witcha can i get witcha',
'Get high get high get high',
'Gettin physical like olivia newt',
'I got the funk flow to make your drawers drop slow',
'Damagin i release the cherry they been through the floor',
'Hah i got the funk flow to make your drawers drop slow',
'Das the more problems we see i dont know what they want from me',
'Nixgas aint got a gun up in your waist please dont shoot up the place',
'I got the funk flow to make your drawers drop slow',
'You nasty boy you nasty',
'Guess thats why they broke and youre so paid uh biggie biggie uhhuh',
'For the mac in the ac d*ck',
'Cause i get witcha can i get witcha',
'Im a bad bad boy',
'Gimme the loot gimme the loot',
'Id probably do real things',
'Beef is when your moms is poppa',
'Sky is the limit and you know that you can have',
'Turn it to die with the wall to me',
'It aint no one but i got a d*ck and puffed me',
'Rip in the mac of the house of paradise',
'I got the funk flow to make your drawers drop slow',
'I got the funk flow to make your drawers drop slow',
'Even all my bronx nixgas nixgas',
'Im a bad bad boy',
'Im a bad bad boy',
'Lunches bone and biggie biggie',
'Why you wanna get with me',
'For the mac in the ac d*ck',
'We dont get witcha can i get witcha',
'Buy the nixga chic we get witcha can i get witcha',
'I got the funk flow to make your drawers drop slow',
'Im a bad bad boy',
'Dont stop Im not finished yet',
'I got the funk flow to make your drawers drop slow',
'At the mariott we go to give me what you need',
'Nixga please be cool but they know we go',
'And i got to split ya',
'Goin somewhere let me to be in icu ya',
'I got the funk flow to make your drawers drop slow',
'Man i got my position and died',
'When i was young in the gray uhhuh',
'Slam larry johnson and your dogs love to die',
'Seen the frick you got a d*ck with the waist please dont shoot up the place',
'Ima know that p**sy many girl',
'Bought the cherokee to the crib unless they bonin',
'Just bone and biggie biggie',
'It aint no one but i got a d*ck and puffed me',
'And i got to split ya',
'The weak or the strong who got it goin on',
'Pussy crusher the game talk your poop grab your gat call your clicks',
'That i killed the spot',
'Im a bad bad boy',
'Can i get witcha can i get witcha',
'Sky is the limit and you know that you can have',
'She know you used to me',
'All the ladies if you hear me',
'Far up in the caribbean white sand my lyrical shift',
'They know what we mean',
'Peeps to cop when i hit my ass',
'Hit the gunshots poop',
'But i dont care what nobody say',
'I got the funk flow to make your drawers drop slow',
'Them nixgaz do real things',
'Whoever i kiss your bum goodnight',
'I got the funk flow to make your drawers drop slow',
'I got the funk flow to make your drawers drop slow',
'Nixgas aint got a gun up in your waist please dont shoot up the place',
'Every blunts in your kitchen screamin eggs and tear',
'The weak or the strong who got it goin on',
'Bitches aint no mother wished they got to split',
'Come on motherfrickers come on',
'frick tae kwon do i get witcha can i get witcha',
'So i dont wanna get witcha can i get witcha',
'The weak or the strong who got it goin on',
'Lookin on the ave Im a man girlfriend',
'Whats goin on',
'And i got to split ya',
'Make the cherokee to the frick to the d*ck',
'Gettin physical like olivia newt',
'Whats goin on',
'And i got to split ya',
'Fulfillin dead wrong',
'By the cherokee to the beds droptop',
'Instead of a mac of the house of girls karl jeans',
'Smokin blunts in your passenger dream',
'Take the beatdown if i bust to the life',
'Real nixgaz do real things',
'Thug one',
'No no notorious',
'Sometimes your words just hypnotize me',
'Make the cherokee to the frick to the d*ck',
'Lean back lean back lean',
'frick tae kwon do i get witcha can i get witcha',
'Sky is the limit and you know that you can have',
'And i got to split ya',
'Im a bad bad boy',
'Im a bad bad boy',
'Shoulda sensamelia can i get witcha can i get witcha',
'Apologies in fact Im sick of blue',
'You nasty boy you nasty',
'Can i get witcha can i get witcha',
'And i got to split ya',
'Im a bad bad boy',
'I got the funk flow to make your drawers drop slow',
'Whyall believable believable',
'The weak or the strong who got it goin on',
'Like i got to be a d*ck and t',
'That i killed the spot',
'Its bone and biggie biggie',
'Think i kiss your bum goodnight',
'Now i dont know what they want from me',
'Mad dead wrong',
'Whats goin on',
'I got the funk flow to make your drawers drop slow',
'Federal is when i was shot',
'Sex is all i get witcha can i get witcha',
'Squeeze your clip hit the right one pass that sandwhiches i got to light one',
'Why you wanna get with me',
'Sing on motherfrickers come on',
'Aint no one and the gray uhhuh',
'Breakin up out a bladder they touched up the grill',
'I got the funk flow to make your drawers drop slow',
'Find the hands in the air if youse a true player',
'So i dont wanna get witcha can i get witcha',
'Im a bad bad boy',
'Jack the nixga but i got a d*ck and puffed me',
'Yall aint no mother wished they got to split',
'I got the funk flow to make your drawers drop slow',
'I got the funk flow to make your drawers drop slow',
'See i dont know what they want from me',
'frick tae kwon do i get witcha can i get witcha',
'Pop in the chamber the streets and the phone',
'Smiles is the drugs baby i got to split',
'Thou larry johnson from the bridge three bricks',
'Before i get witcha can i get witcha',
'Sometimes your words just hypnotize me',
'Spit your game talk your poop grab your gat call your clicks',
'Whats goin on',
'Years is about the block and Im sonning you know that i wont stop',
'Meet the cherokee to the beds droptop',
'Peep the funk flow really though',
'Straight to cop a gun in the house',
'The weak or the strong who got it goin on',
'Keep on pressin on',
'Get high get high get high',
'I got the funk flow to make your drawers drop slow',
'I got the funk flow to make your drawers drop slow',
'Nixgaz is the nixga but i got a d*ck and puffed me',
'I got the funk flow to make your drawers drop slow',
'Im a bad bad boy',
'Why you wanna get with me',
'I got the funk flow to make your drawers drop slow',
'Sky is the limit and you know that you can have',
'You nasty boy you nasty',
'Let me to all my frick is the d*ck',
'Smiles is the drugs baby i got to split',
'Ten years',
'Put the hands in the air if youse a true player',
'I got the funk flow to make your drawers drop slow',
'Just bone and biggie biggie',
'Grab your titties for the funk ill die for the funk',
'Bought the cherokee to the crib unless they bonin',
'To all my queensbridge nixgas nixgas',
'All the ladies if you hear me',
'Mr passes turner she thinks she dont be a glock in my waist',
'Hanging wit the b*****es is the song i sing',
'If you got a gun in your waist please dont shoot up the place',
'Uh i get witcha can i get witcha',
'Long as i got my d*ck but we dont shoot to the door',
'Representing the more problems we see i dont know what they want from me',
'Bleed just like us',
'Man i got my position and died',
'To all my queensbridge nixgas nixgas',
'Now i dont know what they want from me',
'The weak or the strong who got it goin on',
'Tell me why you wanna get with me',
'I got the funk flow to make your drawers drop slow',
'Im a bad bad boy',
'Lets ride lets ride lets ride',
'Lets ride lets ride lets ride',
'Nixga please be cool but they know we go',
'The weak or the strong who got it goin on',
'The weak or the strong who got it goin on',
'Steps just like us',
'Its bone and biggie biggie',
'The weak or the strong who got it goin on',
'One in the chamber the streets that',
'And i got to split ya',
'Bite your hands in the air if youse a true player',
'Stupid the strong who got my d*ck but t',
'Man i got my position and died',
'Swig the wrist buttercrunch',
'Says me for the condom filler',
'Moet yah Im awake man',
'All the ladies if you hear me',
'Pissy nixgaz go from the bridge',
'Then i get witcha can i get witcha',
'We dont get witcha can i get witcha',
'Cristyle no no notorious',
'Mcs rock around my d*ck but the e',
'The weak or the strong who got it goin on',
'Slugs missed ya',
'Oh i dont wanna get witcha can i get witcha',
'Ima know that p**sy many girl',
'Whos the hands in the air if youse a true player',
'All the ladies if you hear me',
'Nia to the mac make the kidneys shift',
'Drop the truck of new york',
'Pimpin up in your caribbean but she was mackin and the hibby you was a lot',
'Im a bad bad boy',
'Shits groups and spending and left your world',
'Fk to all my new nixgas nixgas',
'Oe in the chamber the streets that',
'I got the funk flow to make your drawers drop slow',
'Sky is the limit and you know that you can have',
'N the nixga is chocolate my shift',
'I got the funk flow to make your drawers drop slow',
'What you want nixga what you what you want nixga',
'Willie d havin nightmares of girls',
'For the mac in the ac d*ck',
'Until i get witcha can i get witcha',
'If you got a gun in your waist please dont shoot up the place',
'Cause i get witcha can i get witcha',
'It aint no one but i got a d*ck and puffed me',
'I got the funk flow to make your drawers drop slow',
'And i got to split ya',
'Sky is the limit and you know that you can have',
'Thought i was young in the gray uhhuh',
'What you want nixga what you what you want nixga',
'Nobody gotta die',
'The weak or the strong who got it goin on',
'Yall aint no mother wished they got to split',
'In the cherokee to the beds droptop',
'Im a bad bad boy',
'Still tote a purpose on the bladder and gentlemen with these elevated world',
'Thats the nixga size in these karl kani jeans',
'Told me why you frick doin when i call me when i call me and the scrolls',
'Its bone and biggie biggie',
'She know you used to me',
'If you got a gun in your waist please dont shoot up the place',
'Tell me why you wanna get with me',
'You nasty boy you nasty',
'And i got to split ya',
'Approach me to all my new nixgas',
'Eatin passes turner dinin classes squeezing asses',
'What you want nixga what you what you want nixga',
'Oh i dont wanna get witcha can i get witcha',
'I got the funk flow to make your drawers drop slow',
'I got the funk flow to make your drawers drop slow',
'Free passes screamin aiy papi',
'What you want nixga what you what you want nixga',
'Frank d havin nightmares of a melon of girls',
'Nixga please be cool but they know we go',
'Sex is all i get witcha can i get witcha',
'N the nixga is chocolate my shift',
'Remember i kiss your bum goodnight',
'The weak or the strong who got it goin on',
'I got the funk flow to make your drawers drop slow',
'In the cherokee to the beds droptop',
'Throw your hands in the air if youse a true player',
'Scream you know what you need to do',
'Mc yah Im awake man',
'Loungin at the truck up in your bridge',
'Do you got a gun in your waist please dont shoot up the place',
'When i was young in the gray uhhuh',
'Just bone and biggie biggie',
'I got the funk flow to make your drawers drop slow',
'They know what we mean',
'And i got to split ya',
'Look at the streets to the phone to the door',
'Givin in the club sippin moet in benjamins',
'Cintamelia all you wannah do is touch ya',
'You nasty boy you nasty',
'Just bone and biggie biggie',
'Throw your hands in the air if youse a true player',
'Imma want to die for the paper',
'Hit the gunshots poop',
'To all my queensbridge nixgas nixgas',
'And i got to split ya',
'I got the funk flow to make your drawers drop slow',
'While i just love your flashy ways',
'Oh i dont wanna get witcha can i get witcha',
'If you got a gun in your waist please dont shoot up the place',
'Betcha biggie biggie',
'Youll see youll see',
'That i killed the spot',
'I got the funk flow to make your drawers drop slow',
'Man i got my position and died',
'Bitches aint no mother wished they got to split',
'We dont get witcha can i get witcha',
'All the ladies if you hear me',
'Bring the hands high in a air in your notorous',
'The weak or the strong who got it goin on']

In [31]:
# make sure of length 10 per line
full = ' '.join(RNN_lyrics)
split = full.split()
RNN_lyrics_ten = [split[i*10:(i+1)*10] for i in range(0,int(len(split)/int(10)))]
RNN_ten = [' '.join(RNN_lyrics_ten[i]) for i in range(len(RNN_lyrics_ten))]
len(RNN_ten)

315

In [32]:
qq = map(classify_str, RNN_ten)
list(qq).count('rap')

304

In [33]:
304/315

0.9650793650793651

In [34]:
syl_lyrics = ['Im lookin just living to pay the booty like the',
'Wet em  and live no car  run from police will',
'Before  got the bullets room to the bricks like that',
'Flex scratches the funk baby hear what chorus verse the',
'Loaded in it is the nicest m take her you',
'Latch wit big ladies if you see her sister the',
'Choose to leave it to the train track or black',
'Light another drug to sleep safe up on the average',
'Wherever you know aint over here  when i drop top',
'Box  yall nixgaz back back and change from me swell',
'Begun  to all thick with that hooker off when its',
'Account  so i thought i love to be the back',
'Loop all that good dick size in the relationship grew',
'Yourself when the an rb bitch was poppa aint no',
'Boogey to flip  show they were no telling no dough',
'Biz  oops im doin tonight that shit  and i dont',
'Dome homes i can keep your heart ripped the bread',
'Snitch uhh check it slimmed  hah sicka than the right',
'Supreme in the one thatll pay  who we come you',
'Actors and move trays serve em with the rhyme that',
'La carte with the thriller in your hips ya didnt',
'Pick up  threw that died would be fucked a new',
'Delayed as kris brother with game talk about a big',
'Snoring all my bag yet  be snoring all i might',
'Guns i only give up  even the jeweler and the',
'Legendary nixga ya wet like to feed the flows and',
'Undressin em to side to do two clips and you',
'Blowin up the ballin stops nixga what make our crown',
'High off my feet  cause you come come on come',
'Welchs grape  i hear me i met on my energy',
'Hah one pass that hiphop them they fittin like snoop',
'Need us with me papichulo to die  sex on here',
'It was thinkin about a gs  the weak or do',
'Flame right i see nixgaz on here say im fuckin',
'Fifth of people sho nuff befo i was pussy  but',
'Fake jack boys we was goin down  what you know',
'Layers true fuckin em do when yo  just bone and',
'Thee  one in your kidneys shift here i got the',
'Hydro rock her nose is when you with your shit',
'Try throw em im the rim  got your tim boots',
'Supports this one  off her sister damn boy bad boy',
'Chubby waste of makin strictly back when no more time',
'Crusher black mans mind i dont wanna squirt her  now',
'Jack dempsey would start to cali  i roll like a',
'Ill yo biggie smalls dont get witcha can i drop',
'Singin and if theyll laugh when things born shoulda been',
'Talons loaded in the scrolls  when i fuck time cmon',
'How to feel  what you fuck from behind which wall',
'Rugged as i live without that ever lived the force',
'Eye yo dont have a cab to juggle  make me ',
'Cassette running me close to call it dont know what',
'Cnotes by these cristal  and i up in there wasnt',
'Bathroom tile  steps out your back of me no less',
'Rap attack mr rourke  sure is a brick  lets ride',
'Staff they come on the joness nixgas  dont stop till',
'Cupid  to all my bitch belongs with the gs i',
'Stacks gone when you for mafia to all my shit',
'Shout i never run with  time to scream bam bam',
'Sips on yeah  deja vu the beef no you cant',
'Cleared them in the ave im a better strip  beef',
'West  even though the heart ripped the stash  no need',
'Mug mo bee  killin ya then i dont know you',
'Henny sipper  i dont know that just love ma gat',
'Born sinner the best in the fuck her times  im',
'Back i like bobby  lesson a b k to all',
'Swizz oh thats the mac unloaded  should i get high',
'Backspins to your head and burn when the gangstas thugs',
'Walked them off well be cluckin in her eyeses do',
'Fucked a brother with the fool cause now tell me',
'Drock went  no no gettin shot ya didnt last  im',
'Packin glocks and on the bitch taya ran with jose ',
'Gon get high get it out  oh goddamn credit dead',
'Armed and take em cause money livin so for my',
'It is played fools out on the trey deuce and',
'Glock cocked ms and biggie tell me  i get with',
'Due south  the weak or stiff tongue never said big',
'Half  we just all my mind playin hah i b',
'Swingin swords in from most you what what time  spit',
'Splat  you know your gat call me  gon blast i',
'By the funk  we can be in the mariott we',
'Whiteskinned or two clips at the limit and my dick',
'Milked out here  forget the fuck wit my moms and',
'Moms crotched up and tammy huh huh  beg my mans',
'Feds catch an rb bitch gettin took the lifers groups',
'Sean if i cant even fuckin neck  know my paper',
'Side b side lyrical douches in the lord my name',
'Uno never thought i got his chest  for the back',
'Mac unloaded  intro goes out clear the funk ill go',
'Thatll thrill is goin on and you about this felon',
'Lali like bitch good ass goodnight all ma little visit',
'Tim boots from shaolin island and love me show me',
'Twenty gs up front im with  beef is  chills up',
'Coochies spitting my love of the same hood like death',
'Wid em  we fucked up you heard whos this is',
'Bee baby hear what that good headll do when you',
'Squeezin aes  that shit that we see uh  hit em',
'Ej and gators  just bone and you for the deck',
'Boxers so what the name brand newbian shit list biggie',
'Finna hold more kick in my feet than deep  fucking',
'Steps out the whip it out  pardon my nixga hit',
'Meet again my man is weak or see no problem ',
'Exprained to stop  fuck the fuck it took the e',
'Wars im stressed yo keep on  youll get shot the',
'Deaths when you got that light one else only give',
'Ticket to think shed rather die good night  just keep',
'Vantage came to go blind dont hit the triple beam',
'Buncha nixgas there aint no ends to die  cmon we',
'Still transport with the day no less than yours yeah',
'Pop to dippin  you see them  we dont give when',
'Cost him  check it goin on you that pop bottles',
'Hell  what you are they got love it  the weak',
'Sticks and gators  and i smoke hydro rock on mtv',
'Destroy a few cats wit the ghetto in the pimp',
'Seperately frank her man inf left over two rings and',
'Rate  a tbone steak cheese eggs and fuck all a',
'Leakin like the sex im broke at  lets go off',
'South with them tricks fuck do whatever  but nothin but',
'Late  the hallway got low blow your clip hit the',
'Dogs love you so lay on yeah i gotta die',
'Nia i aint the hardcore wit my faction dont sweat',
'Better be home tomorrow  lets get high with all the',
'Knowing that weed up hands high get your upper lip',
'Champagne when im on yeah i feel one gives me',
'Tallahassee to him half my back lean back back  and',
'Pie  i laugh when i smoked over s cocked ms',
'Mention me for ya  got my gat call me hit',
'Blows was rough very well who was a tutor hah ',
'Gourmet dinners  i had to do to cali got out',
'Sticky green and you are  solo is goin on some',
'Cant you see what a reload like dutches from her',
'Song i got the weed  high off  crystal pops im',
'Figured rock set it this money than yours wack oh',
'Suckin dick  or beneath me to work you had to',
'Shirt  cant we get ya see  may see  skip the',
'Another hons in the old habits two pair of pocket',
'Story as i that dry baggin five nixgas back light',
'Deadly  jack wit the fuck what you knowwwwww where my',
'Upon a stepbystep booklet for me now check it i',
'Testing coke cut it feels good  who you when i',
'Continue to give me milks at they luckand they be',
'Discuss money in the strangest if u need sick of',
'Collected so she aint no mo  all i was in',
'Missile yo back in love its easy to set for',
'Doing time for my hooligans in they stacks in la',
'Hits  shooken up i saw my man hands in your',
'Olde e and taste  the more money than your shit',
'Whyall talkin ayo cmon now  uh damn sir sniff me',
'Freaked em in belize when i hope youre goin on',
'Totin techs than birds to be styling on  what do',
'Valley of rap shit  the benzo she thinks she split',
'Called her you did him that cat all of fuckin',
'Masters we finish it what you with big for my',
'Sicker than cortex singers come to give a bag of',
'Brat  you just splits ya  i went for my nixgaz',
'Lips a tissue stop  just keep on  beef is lick',
'Pockets swell to make one  before all the road to',
'Score with me why christmas missed us talk your chest',
'Drank three hundred shots in the ill shits legit  sex',
'Mall out some shit since the funkdafied cutie wit the',
'Whats the pelvis  to put your gat call it up',
'True fuckin rich the burner was a hundred cash to',
'Bone up with pimps a mess  on yeah  throw all',
'Mass his soul to do  loungin at the fuck up',
'No seeds out in the funk  keep a certified mack',
'Hoodie off the spandex and things  i cock and the',
'Thief  fuck the sleeve red rum its like arnold wondered',
'May be cryin like we be laying with blue drop',
'Usin all my cuban mommy cooking up on back to',
'Peel more  to all leakin out its time  guess thats',
'Wizzard fuck with oil my man see you aint have ',
'Show they pussy whipped nixga that speak to grind like',
'Total  hate to die on my peeps with you do',
'Trips to wait a pickle be sippin moet  come on',
'Linen me  i cant sleep  the seven macs the lord',
'Easy but  get high get witcha can i been shot',
'You dont think about you bought me fill ya to',
'Priest at all get  make up big city i love',
'Position here come the back up  get live for the',
'Swayze bitch if i dont stops nixga please dont know',
'Lovell still get who wine i had been through  what',
'Pops mixed up in the pack the limit and bitches',
'Talking to cali uh and biggie gimme the lord my',
'Ya eye yo fuck around my friends shit  and i',
'Game shakes me big but t  tell him  and taped',
'Asswhippin  ill die now ill put a green like the',
'Homeboy lance armstrong of fact im like a keys so',
'Pop quiz thats what nobody til the ghetto  the stress',
'Obsolete  what what big when frank  the weak from me',
'Personal with a tight dress i took out honey check',
'Use  yeah nixga workin been on here come on the',
'Escape out clear the crib wrap you want that way',
'There was poppa been robbed  its that weed spot back',
'Scanners nixgaz do real hard cause i reach for my',
'Sucked all i bet they sell records like francis mh',
'Onyx and know just ate  nias wanna live for hoes',
'Even though  you too much better yet i gotta get',
'Messin with my dogs love ma little nixga to all',
'Henny and said if i got gin  explode the door',
'Han solo with a asthma attack  looks like them red',
'Each and flee the habit i fighting to squeeze lead',
'Alberta over here tryin not for my cd blast  i',
'Guarantee a gun as i spark ya i dont get',
'Sixteen shots for my sheisty crew be like to fight',
'Stash in love your bank like food tramp  cause i',
'Been shot ya live for the average symps found a',
'Specially if i hope i need on the time to',
'Teary eyed damn boy you not able to know that',
'Guest appear  thats my girl fucked fo keeps n the',
'Slinger bring back to be what you bought a glock',
'Fall in the ladies in they just listen to reverse',
'Trap yes im classic like me  you still wannah do',
'Dark as long gates snatch initial name one more ruthless',
'Mack  i got ready to side to turn your words',
'Dick girl  the weak or stiff tongue or should just',
'Peace to get down your shit thats crazy gs out',
'Sharp bar uh biggie aint spend no comin to giggle',
'Supply the scarf around and biggie let a bunch of',
'Spent the new rappers shouldnt have the fuck around his',
'Fourfifth so take em go you gonna go mase aint',
'Shawn  i can like bitch just for a question what',
'Pipes others use em  cause i can bring back light',
'Jose  hoe money at me what you can like this',
'Rugs thats damn near so you cant you come across',
'Rolls roll in your wig  dreams of all of my',
'System that all i got a nixga what you been',
'Twitchin noses  so dont want to play me to stop',
'Bake up my shirt  time time  nice and youre so',
'Peep the party and thats right  looks like a nixga',
'Loungin black smoking sacks up i cock in this is',
'Callable gats that smoke blunts down to wet me  i',
'Youve been in this whats goin on  respect is not',
'Sweet  see who got stepped on here  your life is',
'Spell my ring  but smalls crib unless they know im',
'Bastard  just bring yo he had no less than biggie',
'Nixga turned around shit too kind of mine aint that',
'Seems up out the likes that nixga  spit your hands',
'Onto dkny uhhuh  i was wise  its more money can',
'Boss hog was young fresh pair of applause for the',
'Wood style dumpin  come on ten i keep on i',
'Ultimate rush the funk ill put it hot  once i',
'Twista and im broke as strong who got down below',
'Feelin of you hear death knockin at the phone mase',
'Pohey  i used to drink brew and pray for tips',
'Karl kani jeans  all philly hoes dough and i cock',
'Big  then she hit the fallout word up on ready',
'Macing yall death  and me all day is been on',
'Legends on back  its real life  and all my wrists',
'Thinking  lets get the gray uhhuh i choke ya shit',
'Fingers in your tim boots and not all my knees',
'Pockets broke and love it  she was a all work',
'Jd i only smoke ls  yes flex who got it ',
'Grinding all up open up to riches  im done i',
'News team  here come for the user  the stress  run',
'Script bust you can count  hah thats how we just',
'Field  so much i hope she kicked all we flew',
'Tossed to go down and heavy d havin fun with',
'Grapes so i dont wanna live it when you to',
'J and roll like i remember back right one  the',
'Actors and still watch me  another hons in the head',
'Stacked was a brother they dont pay the shit splatter',
'Brick  i seen mad at any motherfucker and when i',
'Say what he buy a hydrant  i replace on him',
'Bill about the spandex and this yeah poppa dont care',
'Bring that ass up and bullshit  just the crib unless',
'Vcrs in my fingers in this man when your clicks ',
'Poppin ho thats that nixga its almost there aint mad',
'Ave im so hard with my accountant handles that we',
'Hands in the gall  its the puff daddy  by a',
'Pint drink it feel it goin on the cleanest meanest',
'Fresh out fresh out six rivals spittin fire  way she',
'Rolled up you call me big poppa dont ask rog',
'Dj zhane she setting up my girls boo true player ',
'Mine  cause he had the clits had two rides than',
'Thee  i dont play her mouth  saw me take that',
'Apart break up for me sean combs  call my wrists',
'Hurts  i was rimmed up each and sean combs  all',
'Street life every time i do em i was wise ',
'Of funk ill get a hustle another day no cryin',
'Thys wit us  here we be scheming on the crowbar',
'Thats the shits legit  fuck the deal  when i got',
'Try to motherfuckers as i got g ya  as we',
'Ask nino he wanted with the clips in this willie',
'Official doe i got that weed no more ruthless huh',
'Ripsta nixga wanna tote guns is goin on top sky',
'Found out the big poppa dont shoot em  ask p',
'Ask p leave you got a try  pop to win',
'Trunks bitch keep on the bitch  by the east coast ',
'Others use a ye yea  move from behind which wall',
'Thigh the streets to the th grades and emptied the',
'He came twice caught him  im ready for most of',
'Milks at the flyest its back of your crew  and',
'Shot ya game talk to do aint no trivia  bk',
'Bendin over here we go play me get witcha can',
'Huh can i got the hooptie  to all my side',
'Grime  guess thats right i be what they figure im',
'Sand they know  i fuck up  to all you know',
'To lace these hoes at where do fight  pray and ',
'Swoll on the burner was ya gotta die  is my',
'Big poppa dont want nixga what you heard was in',
'Cuffed lookin cool fool  your life is weak ayo big',
'Fanny spendin  and throw the tech g get some dro',
'Boss of the fuck with the sky  forget the streets',
'Whyd you what im the road to light  they actors',
'Wan test me the willy hitchlin cocked back back in',
'Kelly the shit  sex is gucci bag and im not',
'Imma stay low killer now you swear to feed i',
'Da cause i just hit the scene  game tight  shot',
'Whyall talkin real dogg for the east coast i go',
'Huns in ya imma front do i provide you know',
'Hurting my will show on a bad boy you  oh',
'Kills you to cali  guess thats me walk down phizat',
'Seas vcrs in this robbin spree well make me spendin ',
'Ma little secret why ooh im paid hah  i rap',
'Bleach in they bitch just bring nixgas wanna be what',
'Je and ginger ale  get live no no more  bust',
'Until i live motherfuckers just talk to stop im just',
'Ran with that we out in that you came quicker ',
'Lukes boat  actin as hell  so i have fun  while',
'Strapped like a killin wives and come on hoes clockin',
'Weighters  i see me oh i had to the past',
'Else only smoke the rolexs and pops mixed up in',
'Sofa  and all day  you did where you cant touch',
'Rude  what you what my bitch chose not you up',
'Rapper slash illest turned front of them licks up my',
'Gun and im intrrupted by the house of mine uh',
'Das why im sick of uncut to  my tape around',
'Trep  thank you with the road to visit and my',
'Starters from the same ol flow really dont care if',
'Riot  we dump ashes on pressin on  lights get up',
'Holler if youse a friend bitch with deep so she',
'Endin  you nasty boy what i asked me a big',
'Lies  i knew that cocksucker gs out s in the',
'Dancers  i wish i beat so hard time  throw the',
'Nice dream  give me hip to feed em out  what',
'Wetter than my will probably fuck it goin to his',
'Ron bought me  i see you want to me feel',
'Faction dont think i cant you to see what it',
'War  sold more i thought i love your hands while',
'Cruise up  guess thats why why i ever lived the',
'Honcho  just bring me evil inside the strong who want',
'Mines  let the house on  cant live for a lot',
'As if it  the faggot took nuthin but i got',
'Hutch stick up  mad women from the funk ill die ',
'Blunted up  then fuck with pimps and squeeze it bite',
'Alright when i was  may see  can i got a',
'Bed  sky is wrong i hear me smile on  cmon',
'Singles on  and i wish moms and youre from left',
'Thang like my baby one  hit the gate hungry they',
'Knockers  and now we date  you swung a whole time',
'damn ma i gets rugged crew  thats not to',
'Info for some shit  aint scared of mine  my life',
'Ima send her pissy drunk over this  starters from that',
'Whip it nixga to all from the pen  grab your',
'Tops i have mad at they be friends too many ',
'Dope dealer  if i hope youre fat or get with',
'Leave calm  and party and bring yo hands in the',
'Whats wrong relax and aint the fight ways  we can',
'Ds knew like  dont stop yall my pocket thats how',
'Out like its man  why you to the inside the',
'James todd smith  so what the wall you can i',
'Spock  get live motherfuckers come come suck call him with',
'Howda stack some real bitch named symone  now i get',
'Trippin blamin me jack boys cant tell me into stocks',
'Spendin  plans to get a bag a baby all i',
'Noses  pop duke shoot up bitch like cleats  two keys',
'Five gs a waste  bought the place why go down',
'Legs leave i bought  and hood with my ace boom',
'Serving em  threw that you call me to be the',
'Used to the lyrical carjack make the same ol pimp',
'Mics n blow to live for the tristate  sky is',
'College park  wit the limit and show me when to',
'Killin wives and crowded  id like that thats that i',
'Bedspread red rum  the games we can bang boots and',
'Agreed to you said fuck it goin on the gat ',
'Stay far  no sleepin he was i like i sing',
'Notes than you see  yo they cant hear me homey',
'Relate to give your game talk your name out the',
'Still tustle still tryin to go call your words just',
'Lost a fuckin hour glass is the fame start blastin ',
'Name drew played fools bash fools dat nixga start  whos',
'Smoke some more money in raps what must be handlin',
'Beotch fuck the more time losers mad bitches who says',
'Wall and why  cmon yall live for the fucking chase',
'Abuser  of the sky  she got it goin on in',
'Befo i hope i dont believe in the drugs to ',
'Rehearsed it means to flip the tec to cali cali',
'Whip legend coupe gold teeth milo cut nixga was short',
'Hypnotize  when i swear to snatch a set for the',
'Boys cant happen dont get high get it when you',
'Leaking out  then she got the shits with that you',
'Silly cat with me what the sequel  and another drug',
'Smokin spliffs we all my nixga wit me big poppa',
'Beacon  put in the corner my head and im glad',
'Putcho nose bleed  i used to big ayo big be',
'Album back of fuckin but if they say  you cut',
'Looks out and gums all you can i love potion',
'Spill  them cats we give you while my energy is',
'Split it would you know my vest  kill em like',
'Jake come on stage the streets  she cryin at all']


In [35]:
len(syl_lyrics)

399

In [36]:
pp = map(classify_str, syl_lyrics)
list(pp).count('rap')

366

In [37]:
366/399

0.9172932330827067